In [3]:
import pandas as pd
import os

from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

import nltk
from nltk.corpus import stopwords
from nltk.stem import *


In [4]:
path = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/spam-filtering"

filenames = os.listdir(path)
videos = [pd.read_csv(path + '/' + filename, sep=',') for filename in filenames]

dataset = pd.concat(videos)

In [5]:
dataset.sample(frac=1).head(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
334,LneaDw26bFuhuiZ8uX6C-qYLIsOFj9BIWtKWtCz870c,Lizzy Molly,NaN,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,1
60,z13hxdr5uvbsintfz22uddk4imbkcp20h,Mattkrewe,2015-05-25T20:36:42.681000,i was born in the wrong generation﻿,0
66,z13azdiwulmkf5kmr22cwbpq1zrntfxby04,Alexander Solli,2015-05-24T12:32:18.770000,My uncle said he will stop smoking if this com...,1
351,z13eenia5rqwhvmyw04cgt441wapvbkakzo0k,faisal joy,2015-05-21T18:24:19.018000,Hi I am from bangladesh 💜﻿,0
240,z12mhbmbhlv5jbi1n231chnqtrmkjrenc,MrSlowGhost,2014-10-21T15:30:56,It should be illegal to be this goodlooking as...,0
152,z13jwbvxvoqje10yg04ccbx5lna5w5rr0tg0k,EraserMan2,2015-05-19T00:46:08.243000,why does the world not shuffle???﻿,0
227,z13wj1pgwlfjjjn4d04cilo5nwnhsbdavzg0k,Tee Tee,2014-11-07T20:16:51,Loool nice song funny how no one understands (...,0
264,z135cxegbufxdviso23kgn4rqmbvwxyqp04,Spencer Clark,2014-11-08T05:29:26,If you pause at 1:39 at the last millisecond y...,0
184,_2viQ_Qnc68YhzlKOdq3h2y3v8TCi0RQSIFzQP8hltY,Derek Moya,2013-10-21T05:47:26.457000,You guys should check out this EXTRAORDINARY w...,1
366,z13hxfdrqom5clnaa04cgx5hkpqjexhh3wk0k,DERRICK HANFORD,2014-12-05T22:52:29.316000,White people are going extinct for more inform...,1


In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
stop_words = stopwords.words("english")
vectorizer = CountVectorizer(stop_words=stop_words)

In [8]:
Y = dataset["CLASS"].tolist()

In [9]:
wnl = WordNetLemmatizer()

In [12]:
def hypertune_svm(x, y):
    model = svm.SVC()
    kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    C = [1000, 100, 50, 10, 1.0, 0.1, 0.01]
    gamma = ['scale']
    degree = range(1,11)
    # define grid search
    grid = dict(kernel=kernel,C=C,gamma=gamma, degree=degree)
    
    # model = LogisticRegression()
    # # solvers = ['newton-cg', 'lbfgs', 'liblinear']
    # solvers = ['saga', 'liblinear']
    # penalty = ['l1', 'l2']
    # c_values = [100, 10, 1.0, 0.1, 0.01]
    # # define grid search
    # grid = dict(solver=solvers, penalty=penalty, C=c_values)
    
    cv = KFold(n_splits=10)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='precision', error_score=0)
    grid_result = grid_search.fit(x, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [13]:
X_lem = vectorizer.fit_transform(dataset["CONTENT"].apply(wnl.lemmatize))
hypertune_svm(X_lem,Y)

Best: 0.986799 using {'C': 1.0, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}
0.939185 (0.045790) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.888437 (0.074375) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
0.978323 (0.022249) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.939185 (0.045790) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
0.888437 (0.074375) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
0.977115 (0.027137) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.939185 (0.045790) with: {'C': 1000, '

## Bags of words with stemming

In [73]:
stemmer = PorterStemmer()

In [124]:
X_stem = vectorizer.fit_transform(dataset["CONTENT"].apply(stemmer.stem))
hypertune_svm(X_stem,Y)

Best: 0.986767 using {'C': 1.0, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}
0.940140 (0.045255) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.886586 (0.068650) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
0.978209 (0.022405) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.940140 (0.045255) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
0.886586 (0.068650) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
0.977267 (0.026840) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.940140 (0.045255) with: {'C': 1000, '

In [120]:
model = svm.SVC(kernel='rbf', C=10, gamma='scale', degree=2)
scores_acc = cross_val_score(model, X_lem, Y, cv=10, scoring='accuracy')
scores_prec = cross_val_score(model, X_lem, Y, cv=10, scoring='precision')

In [121]:
scores_acc.mean(), scores_prec.mean()

(0.9488984824699109, 0.950040593728599)

In [14]:
path_to_results = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/researches/spam-filtering.tsv"
results = pd.read_csv(path_to_results, sep='\t')

results

,technique,kernel,C,gamma,degree,precision
0,lemmatization,linear,1000.00,scale,-,0.94113
1,lemmatization,poly,1000.00,scale,1,0.94113
2,lemmatization,rbf,1000.00,scale,-,0.93919
3,lemmatization,sigmoid,1000.00,scale,-,0.88844
4,lemmatization,linear,1000.00,scale,-,0.94113
...,...,...,...,...,...,...
555,stemming,sigmoid,0.01,scale,-,0.61341
556,stemming,linear,0.01,scale,-,0.98234
557,stemming,poly,0.01,scale,10,0.95500
558,stemming,rbf,0.01,scale,-,0.73248
